In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import openpyxl

plt.style.use(["bmh"])

file_name = "./tests/@CN_OGX-101.xlsx"

dataframe = pd.read_excel(
    file_name,
    skiprows=0,
    decimal=",",
    sheet_name="LONGÁ",
)

dataframe

In [ ]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt

# Dados dos pontos-chave de probabilidade do
TOC_TN_ARTIGO = np.array(
    [4, 4, 4, 4, 10, 10, 10, 10, 100, 100, 100, 100]
)  # Lista/array dos valores de TOC:TN dos pontos-chave
C13CORG_ARTIGO = np.array(
    [-10, -22, -25, -34, -10, -22, -25, -34, -10, -22, -25, -34]
)  # Lista/array dos valores de δ13Corg dos pontos-chave
D15N_ARTIGO = np.array(
    [12, 3, 0, -12, 12, 3, 0, -12, 12, 3, 0, -12]  # MUDAR
)  # Lista/array dos valores do terceiro variável

PROBABILIDADE = np.array(
    [0, 10, 20, 30, 20, 30, 40, 50, 90, 95, 98, 100]
)  # Lista/array dos valores de probabilidade dos pontos-chave

# Dados das suas amostras
amostra_TOC_TN = dataframe[
    "TOC/TN"
].values  # Lista/array dos valores de TOC:TN das suas amostras
amostra_C13CORG = dataframe[
    "d13C"
].values  # Lista/array dos valores de δ13Corg das suas amostras
amostra_D15N = dataframe["d15N"].values  # Lista/array dos valores do terceiro variável

# Sample data
X = np.column_stack((TOC_TN_ARTIGO, C13CORG_ARTIGO, D15N_ARTIGO))
y = PROBABILIDADE

# Define the kernel
kernel = C(1.0, (1e-3, 1e3)) * RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e2))

# Create the Gaussian Process model
gp = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)

# Fit to the data
gp.fit(X, y)

# Predict on new data
X_new = np.column_stack((amostra_TOC_TN, amostra_C13CORG, amostra_D15N))

# Handle NaN values by imputing them
imputer = SimpleImputer(strategy="mean")
X_new_imputed = imputer.fit_transform(X_new)

# Predict with the imputed data
y_pred, sigma = gp.predict(X_new_imputed, return_std=True)

amostra = 100 - y_pred

print(y_pred)

In [ ]:
# USANDO O MÉTODO ANTIGO
from pykrige.ok import OrdinaryKriging

# Grade de valores para interpolação
art_toc_tn_grid = np.linspace(0, 100, 1001)  # Valores de TOC:TN para a grade
art_c13corg_grid = np.linspace(-34, -10, 241)  # Valores de δ13Corg para a grade

# Kriging
OK = OrdinaryKriging(
    TOC_TN_ARTIGO,
    C13CORG_ARTIGO,
    PROBABILIDADE,
    variogram_model="gaussian",
)
z, ss = OK.execute("grid", art_toc_tn_grid, art_c13corg_grid)

# Aplicação dos dados das suas amostras na grade interpolada
amostra_probabilidade, ss = OK.execute("points", amostra_TOC_TN, amostra_C13CORG)

print(np.shape(z))
print(np.shape(amostra_probabilidade))

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(12, 8), sharey=True)
fig.suptitle("OGX-101", fontsize=16)
# Personalização do primeiro gráfico
axes[0].plot(
    amostra_probabilidade,
    dataframe["TVDSS"],
    "s-",
    label="TOMI",
    color="blue",
    linestyle="--",
)
axes[0].set_xlim(0, 100)
axes[0].set_title("Índice TOMI", fontsize=14)
axes[0].set_xlabel("Índice (%)", fontsize=12)
axes[0].legend()
axes[0].grid(True)

# Personalização do segundo gráfico
axes[1].plot(
    amostra,
    dataframe["TVDSS"],
    "s-",
    label="TOMI",
    color="green",
    linestyle="-.",
)
axes[1].set_xlim(0, 100)
axes[1].set_title("TOMI - d15N", fontsize=14)
axes[1].set_xlabel("Índice (%)", fontsize=12)
axes[1].legend()
axes[1].grid(True)

# Personalização do terceiro gráfico
axes[2].plot(
    dataframe["TOC/TN"],
    dataframe["TVDSS"],
    "s-",
    label="TOC/TN",
    color="red",
    linestyle="-",
)
axes[2].set_title("TOC/TN", fontsize=14)
axes[2].set_xlabel("TOC/TN", fontsize=12)
axes[2].legend()
axes[2].grid(True)

# Personalização do quarto gráfico
axes[3].plot(
    dataframe["d13C"],
    dataframe["TVDSS"],
    "s-",
    label="δ13C",
    color="purple",
    linestyle=":",
)
axes[3].set_title("δ13C", fontsize=14)
axes[3].set_xlabel("δ13C", fontsize=12)
axes[3].legend()
axes[3].grid(True)

# Personalização do quinto gráfico
axes[4].plot(
    dataframe["d15N"],
    dataframe["TVDSS"],
    "s-",
    label="δ15N",
    color="orange",
    linestyle="--",
)
axes[4].set_title("δ15N", fontsize=14)
axes[4].set_xlabel("δ15N", fontsize=12)
axes[4].legend()
axes[4].grid(True)

# Configuração do rótulo comum do eixo y
fig.text(0.04, 0.5, "TVDSS (m)", va="center", rotation="vertical", fontsize=12)

# Ajuste do layout
plt.tight_layout(rect=[0.05, 0, 1, 1])
plt.show()